# per ogni input controllare che l'ufficio sia presente nella lista dei codici uffici

#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text:  # Check if the link is present in the whole body
                    compila = 'si'



                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [3]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result=''
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        rep = str(numero) + ' - ' + str(anno)
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': ufficio,
                            'natura': natura,
                            'ente': ente,
                            'repertorio': rep,
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number} , ente : {ente} , natura : {natura} , debitore : {debitore}')
        print(f"Error occurred: {e}")
    return results


#### sistemiamo l'input prima di mandare lo scaper 

In [4]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
today = date.today()


In [5]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [6]:
input = pd.read_excel('Input/Raw/perimetro_03_06.xlsx')

In [7]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [8]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [9]:
input = input.drop_duplicates()

In [10]:


input = clean(input)
#input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
#input = input.drop(columns=['RICERCA IQERA'])

In [11]:


mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',  # forse
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'"
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input
input = input.drop_duplicates()

In [12]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [13]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

input = input.drop_duplicates()

In [14]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [15]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [16]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)


In [17]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [18]:
input = input.drop(columns={'category'})
input['code_ente'] = input['code_ente'].apply(tuple)
input = input.drop_duplicates()
input['code_ente'] = input['code_ente'].apply(list)

In [20]:
input.to_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',index=False)

In [19]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,nr,year
0,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",4693,2020
1,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1236,2020
3,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",4693,2020
4,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1236,2020
6,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",4609,2020
...,...,...,...,...,...,...,...,...,...,...,...
5812,XUB385075,BUCCIONE PIERO,tribunale,chieti,Sirio,monitoria,TAH,chieti ut dpch,[001],1001,2023
5814,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJ3,civitavecchia ut dprm2,[001],193,2023
5816,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJF,civitavecchia - ante 01.02.2015 ut dprm1,"[001, 004, 005, 006, 007, 011]",193,2023
5818,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",4962,2021


In [58]:
# new_df = input[2000:2500]

new_df = input[3300:]
# new_df = input[((input['Debitore'] == 'NOTARPIETRO MARIA GRAZIA') | (input['Debitore'] == 'ABBATE MICHELE')) &
#                    ((input['code_office'] == 'TET') | (input['code_office'] == 'TZL'))]

results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], 
                                    anno=row['year'],
                                    numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

Risultati in un df

In [59]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,PAGANI UT DPSA,DECRETO INGIUNTIVO n. 163 del 2022,TRIB.NOC.INFERIORE,612/22 - 2022,ALVINO S.R.L.,-,IN ATTESA DI PAGAMENTO,si
1,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,002,163 - 2022,-,-,-,no
2,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,003,163 - 2022,-,-,-,no
3,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,007,163 - 2022,-,-,-,no
4,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,009,163 - 2022,-,-,-,no
...,...,...,...,...,...,...,...,...,...,...
1633,XUB386298,PESCE MAURO,TAU,AV,001,4962 - 2021,-,-,-,no
1634,XUB386298,PESCE MAURO,TAU,AV,005,4962 - 2021,-,-,-,no
1635,XUB386298,PESCE MAURO,TAU,AV,006,4962 - 2021,-,-,-,no
1636,XUB388609,MERLINO TETTI S.R.L.,CREMA UT DPCR,DECRETO INGIUNTIVO n. 566 del 2023,GIUDICE DI PACE DI CREMA,59 - 2024,SIRIO NPL S.R.L.,MERLINO TETTI S.R.L. UNIPERSON,IN ATTESA DI PAGAMENTO,si


In [60]:
cliente = input[['NR_Contratto','Cliente']]
df = pd.merge(df,cliente,how='left', left_on='contratto',right_on='NR_Contratto' )
df.drop(columns=['NR_Contratto'], inplace=True)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
0,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,PAGANI UT DPSA,DECRETO INGIUNTIVO n. 163 del 2022,TRIB.NOC.INFERIORE,612/22 - 2022,ALVINO S.R.L.,-,IN ATTESA DI PAGAMENTO,si,Wind
1,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,002,163 - 2022,-,-,-,no,Wind
2,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,003,163 - 2022,-,-,-,no,Wind
3,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,007,163 - 2022,-,-,-,no,Wind
4,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,TEX,DI,009,163 - 2022,-,-,-,no,Wind
...,...,...,...,...,...,...,...,...,...,...,...
3113,XUB386298,PESCE MAURO,TAU,AV,001,4962 - 2021,-,-,-,no,Sirio
3114,XUB386298,PESCE MAURO,TAU,AV,005,4962 - 2021,-,-,-,no,Sirio
3115,XUB386298,PESCE MAURO,TAU,AV,006,4962 - 2021,-,-,-,no,Sirio
3116,XUB388609,MERLINO TETTI S.R.L.,CREMA UT DPCR,DECRETO INGIUNTIVO n. 566 del 2023,GIUDICE DI PACE DI CREMA,59 - 2024,SIRIO NPL S.R.L.,MERLINO TETTI S.R.L. UNIPERSON,IN ATTESA DI PAGAMENTO,si,Sirio


Righe con risultati

In [61]:
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
0,WN520157914_22082019,KIKIRRUS CATERING BANQUETING S R L,PAGANI UT DPSA,DECRETO INGIUNTIVO n. 163 del 2022,TRIB.NOC.INFERIORE,612/22 - 2022,ALVINO S.R.L.,-,IN ATTESA DI PAGAMENTO,si,Wind
12,WN521892354_10042019,SAPI S R L,ROMA 2 - AURELIO UT DPRM1,DECRETO INGIUNTIVO n. 9094 del 2020,TRIBUNALE CIVILE ROMA,10474/20 - 2020,FORME E COLORI SRL,RSI RESIDENZE SVILUPPO ITALIA,ATTO REGISTRATO,no,WIND
54,WN525011924_10042019,LA BANDA BASSOTTI PET S R L,SAVONA UT DPSV,DECRETO INGIUNTIVO n. 81 del 2022,TRIBUNALE DI SAVONA,171 - 2022,COND.VILLE LE GINESTRE,ROSA,ATTO REGISTRATO,no,Wind
58,WN525092464_12122019,PIMPINELLA TRANSPORT S R L,LATINA UT DPLT,DECRETO INGIUNTIVO n. 1722 del 2020,TRIBUNALE DI LATINA,903/21 - 2020,DAPIT GIUSEPPE SRL,FERRANTE,EMESSO ARTICOLO DI RUOLO,no,Wind
158,WN526105450_22082019,VERAJ DI AMBROSIO GAETANO,NAPOLI 3 UT DPNA2,DECRETO INGIUNTIVO n. 791 del 2020,TRIBUNALE DI NAPOLI,1255 - 2020,BANCA SISTEMA S.P.A.,COMUNE DI NAPOLI,ATTO REGISTRATO,no,WIND
378,WN534140628_30102018,C F SERVICE SRLS,CHIETI UT DPCH,DECRETO INGIUNTIVO n. 457 del 2019,TRIBUNALE CHIETI,790 - 2019,SIROLLI,-,ATTO REGISTRATO,no,WIND
574,WN538430434_10042019,ING SALVATORE FUCCI,ROMA 2 - AURELIO UT DPRM1,DECRETO INGIUNTIVO n. 9091 del 2020,TRIBUNALE CIVILE ROMA,166 - 2021,IFIS NPL INVESTING S.P.A.,FERNANDO MANCHIA,ATTO REGISTRATO,no,WIND
640,WN541833476_22082019,JORDAN GROUP SRL,CATANIA UT DPCT,DECRETO INGIUNTIVO n. 1767 del 2020,TRIBUNALE DI CATANIA,4864/20 - 2020,CONCESSIONARIA VEICOLI INDUSTR,LOGIST TRASPORTI SRL,ATTO REGISTRATO,no,WIND
680,WN544438189_12122019,GEA GESTIONI S R L,GENOVA 1 UT DPGE,DECRETO INGIUNTIVO n. 585 del 2021,TRIBUNALE DI GENOVA,C433 - 2021,ARATA,ALESSANDRA MAZZOTTA,ATTO REGISTRATO,no,Wind
690,WN545829204_22082019,TE S R L,MILANO 1 UT DPMI1,DECRETO INGIUNTIVO n. 6901 del 2020,TRIB. CIVILE ORDINARIO,10831 - 2020,KEY ONE S.R.L. - IN LIQUIDAZIO,MACRI SRL,STAMPATO AVVISO DI LIQUIDAZIONE,no,WIND


Guardare i nomi in format 'name +1'

In [62]:
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
2320,XSL99109530,SILVESTRI MICHELE,PIACENZA UT DPPC,DECRETO INGIUNTIVO n. 781 del 2023,GIUDICE DI PACE DI PIACENZA,122/2024 - 2024,MARATHON SPV S.R.L. + 1,MINOJA FABIO,IN ATTESA DI PAGAMENTO,si,Lex


Toglie il +1 a chi lo ha

In [64]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()


In [65]:
# df = pd.DataFrame(results.explode().tolist())
df.to_excel(f'Output/{str(today)}_output_totale_fine_v1.xlsx',index=False)


In [66]:
mask = df.apply(lambda row: (row['Cliente'] in row['attore']) or (row['attore'] in row['Cliente']) or (row['convenuto'] in row['debitore']), axis=1)

df[mask].to_excel(f'Output/{str(today)}_esiti_positivi_fine_v1.xlsx',index=False)